# JPA Scattering Matrix Measurement

In this notebook I develop the code for running a frequency sweep scattering experiment on the JPA. The purpose of this experiment is to develop the **Scattering Matrix**, i.e., the reflected frequencies for each injected frequency signal. 

- **Signal frequency set** : $\{4.100, 4.101, \dots, 4.300\}$ GHz ;  **Signal Amplitude**: 0.04
- **Pump frequency set** : $8.4$ GHz ; Pump Amplitude : 0.1 FS
- **Output frequency set** : $\{4.100, 4.101, \dots, 4.300\}$ GHz : Identical to Signal set






In [172]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys
import math
import presto
from presto import lockin, utils, hardware
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode

## 1. Format for saving measurement data

In [173]:
# Save attributes function
def save_attr(folder, file, sample_, myrun_, myrun_attrs_):
    if not os.path.isdir(folder):
        os.makedirs(folder)
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        run_str = f"{myrun_}"
        # Assicurati che il gruppo esista
        if run_str not in savefile:
            savefile.create_group(run_str)
        for key, val in myrun_attrs_.items():
            savefile[run_str].attrs[key] = val
    print("Saved run attributes.")

# Save data function
def save_data(folder, file, sample, myrun, freq, freq_pump, pump_pwr, pump_phase, df, usb_arr):
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # Open the save file (.hdf5) in append mode
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        # String as handles
        freq_data_str = "{}/freq comb".format(myrun)
        freq_pump_data_str = "{}/freq pumps".format(myrun)
        pump_pwr_data_str = "{}/pump pwr sweep".format(myrun)
        pump_phase_data_str = "{}/pump phase sweep".format(myrun)
        df_data_str = "{}/df".format(myrun)
        usb_data_str = "{}/USB".format(myrun)

        # Write data to datasets
        savefile.create_dataset(freq_data_str, (np.shape(freq)),
                                dtype=float, data=freq)
        savefile.create_dataset(freq_pump_data_str, (np.shape(freq_pump)),
                                dtype=float, data=freq_pump)
        savefile.create_dataset(pump_pwr_data_str, (np.shape(pump_pwr)),
                                dtype=float, data=pump_pwr)
        savefile.create_dataset(pump_phase_data_str, (np.shape(pump_phase)),
                                dtype=float, data=pump_phase)
        savefile.create_dataset(df_data_str, (np.shape(df)),
                                dtype=float, data=df)
        savefile.create_dataset(usb_data_str, (np.shape(usb_arr)),
                                dtype=complex, data=usb_arr)

        # Write dataset attributes
        savefile[freq_data_str].attrs["Unit"] = "Hz"
        savefile[freq_pump_data_str].attrs["Unit"] = "Hz"
        savefile[pump_pwr_data_str].attrs["Unit"] = "fsu"
        savefile[pump_phase_data_str].attrs["Unit"] = "rad"
        savefile[df_data_str].attrs["Unit"] = "Hz"
        savefile[usb_data_str].attrs["Unit"] = "fsu complex"

## 2. Setting up the measurement parameters for Presto

In [174]:
### MEASUREMENT SETUP ###

# Filename based on timestamp of experimental run
Ym_str = time.strftime("%Y-%m")                                 # Get year and month
meas_type   = 'Scattering'                                      # Measurement type
save_folder = r'I:/LKiPA-Data/{}/{}'.format(Ym_str, meas_type)  # Create folder for current month and measurement type
myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")                # Save experimental run for each timestamp
save_file   = r"{}.hdf5".format(myrun)                          # Save data in hdf5 file for current run

sample      = 'LKIPA'
atten       = 80
temperature = 0.0096 # Take latest value


# Network settings for Presto Hardware
ADDRESS = '130.237.35.90'   # IP Address
PORT    = 42873             # TCP Port
Box     = 'Presto DELTA'    # Model Identifier

# Port Assignments
input_port   = 5 # Output from LKIPA (to be measured)
output_port  = 8 # Signal to LKIPA (vacuum for correlation experiments)
flux_port    = 2 # Pump frequency comb 
bias_port    = 2 # DC Bias for optimal operating point of JPA (based on dc bias-resonant frequency curve)

# Pseudorandom noise (only when working with small amplitudes)
dither = False

# DAC current
DAC_CURRENT  = 32_000  # µA : Max current allowed for DC Bias 

# DC bias value
bias_val     = 3.0   # FS  From calibration  4.0 V : f0= 4.401 GHz  

In [175]:
### MEASUREMENT PARAMETERS ###

# NCO frequencies (Numerically Controlled Oscillator) : Digital Local Oscillator
fNCO = 4.31e9        # NCO frequency for output comb and signal
fNCO_pump = 8.6e9   # NCO frequency for pump comb
_df_sweep = 50e3  # Sweep resolution = 1 MHz

# Number of pixels
_Npix = 5_000
Navg = 1                                 # No averaging
Nskip = 0                                # Number of pixels we discard


## SIGNAL PARAMETERS ##
# Signal output amplitude list from Presto [FS] (Never exceed 0.05FS for OUTPORT!!
sig_amp= 0.1
sig_amp_arr = np.array([sig_amp])   
sig_amp_num = len(sig_amp_arr)           # = 1 for no signal amplitude sweep

# Signal Frequency list
sig_freq_num = 21               # Number of frequencies of the frequency comb
sig_freq_step = _df_sweep                # 1 MHz
sig_freq_centre = 4.404e9                # center         

## RF PUMP PARAMETERS ##
# Pump frequency in Hz
_fp_center = 2 * sig_freq_centre         # Single pump at 2*w_0 = 8.849711134 GHz

# Number of pumps
pump_freq_arr = np.array([-4, 0, 4])
pump_freq_num = len(pump_freq_arr)
pump_freq = _fp_center                  # Single pump at 2*w_0 = 8.4 GHz

# Pump amplitude sweep (FS units between 0 and 1)
pump_amp = 0.1
# Single pump amplitude
pump_amp_num = 1

# Pump 1 phase sweep
nr_pump_phases = 1
pump_phase_arr = np.linspace(0, 2 * np.pi, nr_pump_phases)

# Propagation Phase Drift
input_phases = np.zeros(sig_freq_num)
dPhiDrift_df = 1.8869636024e-6            # rad/Hz - Measured Drift in phase per Hz

## 3. Instantiate ```lockin``` mode and **RUN** experiment

In [176]:
# Instantiate lockin device
with lockin.Lockin(address=ADDRESS,
                   port=PORT,
                   adc_mode=AdcMode.Mixed,
                   adc_fsample=AdcFSample.G2,
                   dac_mode=[DacMode.Mixed04, DacMode.Mixed02, DacMode.Mixed02, DacMode.Mixed02],
                   dac_fsample=[DacFSample.G10, DacFSample.G6, DacFSample.G6, DacFSample.G6],
                   ) as lck:
    # Start timer
    t_start = time.strftime("%Y-%m-%d_%H_%M_%S")

    # Print Presto version
    print("Presto version: " + presto.__version__)

    # DAC current
    lck.hardware.set_dac_current(output_port, DAC_CURRENT)

    # Initialise data array
    usb_arr = np.zeros((sig_freq_num, sig_freq_num), dtype=complex)

    # Bandwidth array and frequency combs arrays (to save): Although the arrays can be hardcoded as 1x1 arrays, this format allows for code reuse.
    df = 0
    fs_comb_arr = np.zeros((sig_freq_num))
    fp_center = 0

    # Set DC bias value
    lck.hardware.set_dc_bias(bias_val, bias_port)
    lck.hardware.sleep(1.0, False) # Let the system reach steady state


    # Create output group for the pump
    og_pump = lck.add_output_group(ports=flux_port, nr_freq=pump_freq_num)
    og_pump.set_amplitudes(pump_amp * np.ones(pump_freq_num))    # Set amplitude of pump = 0.04 FS
    og_pump.set_phases(phases=0.0,
                  phases_q=-np.pi / 2)
    
    # Create output group for the signal
    og = lck.add_output_group(ports=output_port, nr_freq=sig_freq_num)
    og.set_amplitudes(sig_amp_arr)
    og.set_phases(phases=0.0,
                  phases_q=-np.pi / 2)
    
    # Create input group for measurement
    ig = lck.add_input_group(port=input_port, nr_freq=sig_freq_num)

    # Apply settings and let system relax
    lck.apply_settings()
    lck.hardware.sleep(1e-4, False)

    ### RUN EXPERIMENT ###
    with tqdm(total=(pump_amp_num * nr_pump_phases * sig_freq_num ** 2), ncols=80) as pbar:        
        # Set measurement comb
        _fs_center = sig_freq_centre - fNCO

        # Tune center frequency
        _, df = lck.tune(_fs_center, _df_sweep)
        
        # Signal comb
        fs_comb = df * np.arange(1, sig_freq_num+1, 1)

        # Set NCO Frequencies
        fNCO = sig_freq_centre - ((sig_freq_num +1 ) / 2) * df
        fNCO_pump = 2 * fNCO

        # Set tuned pump frequency
        fp_center = 2 * sig_freq_centre - fNCO_pump
        fp_comb = fp_center + pump_freq_arr * 2 * df

        # Configure mixer for the JPA pump
        lck.hardware.configure_mixer(
            freq=fNCO_pump,
            out_ports=flux_port,
            sync=False,
            )

        # Configure mixer just to be able to create output and input groups
        lck.hardware.configure_mixer(
            freq=fNCO,
            in_ports=input_port,
            out_ports=output_port,
            sync=True
            )

        # Set df
        lck.set_df(df)

        # Set tuned pump frequency
        og_pump.set_frequencies(fp_comb)

        # Set input phases
        input_phases[0]=0.0
        for i in range(1, sig_freq_num):
            input_phases[i] = input_phases[i-1] - dPhiDrift_df * df  # the minus work
        input_phases_q =  input_phases - np.pi /2 
        ig.set_phases(phases=input_phases.copy()) 

        # sweep each signal frequency
        for freq_idx, freq_val in enumerate(fs_comb):
            # Set Signal (probe) Frequency
            og.set_frequencies(freq_val) # signal
            for ig_idx, ig_freq in enumerate(fs_comb):
                ig.set_frequencies(ig_freq)
                lck.apply_settings()    
                
                # get scattering data
                Npix = math.floor(_Npix)
                data = lck.get_pixels(Npix + Nskip, summed=False, nsum=Navg)
                freqs, pixels_i, pixels_q = data[input_port]

                # Convert a measured IQ pair into a low/high si
                # deband pair
                LSB, HSB = utils.untwist_downconversion(pixels_i, pixels_q)
                usb_arr[freq_idx, ig_idx] = np.mean(HSB)

                # Update progress bar
                pbar.update(1)

        # Mute outputs at the end of the sweep
        og_pump.set_amplitudes(0)
        og.set_amplitudes(0.0)
        lck.apply_settings()
        lck.hardware.set_dc_bias(0.0, bias_port)

    # Stop timer
    t_end = time.strftime("%Y-%m-%d_%H_%M_%S")

    # Create dictionary with attributes
    myrun_attrs = {
        "Meas": meas_type,
        "Instr": Box,
        "T": temperature,
        "Sample": sample,
        "att": atten,
        "4K-amp_out": 42,
        "RT-amp_out": 41,
        "RT-amp_in": 0,
        "nr_sig_freqs": sig_freq_num,
        "nr_pump_freqs": 1,
        "fp_center": fp_center + fNCO_pump,
        "amp_sig": sig_amp_arr,
        "DC bias": bias_val,
        "Npixels": Npix,
        "Naverages": Navg,
        "Nskip": Nskip,
        "Dither": dither,
        "t_start": t_start,
        "t_meas": t_end,
        #"Script name": os.path.basename(__file__),
        }
    
    # Save data
    save_data(
        save_folder, 
        save_file, 
        sample, 
        myrun, 
        fs_comb + fNCO, 
        fp_center + fNCO_pump, 
        pump_amp,
        pump_phase_arr, 
        df, 
        usb_arr
        )
    
    print('Data Saved')


Presto version: 2.16.0


100%|█████████████████████████████████████████| 441/441 [01:07<00:00,  6.57it/s]

Data Saved
